In [ ]:
!pip install -qU accelerate transformers bitsandbytes peft datasets sentencepiece tqdm huggingface evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import login
login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
from peft import PeftModel, PeftConfig
import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print("Loading models once...")

# Model 1: LLaMA3 Genre Classifier
llama_model_id = "akshayparam/llama3-genre-classifier"
llama_tokenizer = AutoTokenizer.from_pretrained(llama_model_id)
llama_model = AutoModelForCausalLM.from_pretrained(llama_model_id).to(device)

# Model 2: RoBERTa Genre Classifier
# roberta_model_id = "akshayparam/RoBERTa_LGb-lyrics-genre-classifier"
roberta_model_id = "akshayparam/RoBERTa-LGb-lyrics-genre-classifier2"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_id)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_model_id).to(device)
roberta_id2label = roberta_model.config.id2label

# Model 3: TinyLLaMA Chord Generator
chord_model_id = "akshayparam/T2_tinyllama_chordgen"
chord_config = PeftConfig.from_pretrained(chord_model_id)
chord_base_model = AutoModelForCausalLM.from_pretrained(
    chord_config.base_model_name_or_path,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)
chord_model = PeftModel.from_pretrained(chord_base_model, chord_model_id)
chord_tokenizer = AutoTokenizer.from_pretrained(chord_config.base_model_name_or_path)
chord_tokenizer.pad_token = chord_tokenizer.pad_token or chord_tokenizer.eos_token

print(" Models loaded successfully!\n")

Loading models once...


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

 Models loaded successfully!



In [ ]:
# MODEL 1: LLaMA3 Genre Classifier

def classify_llama3(lyrics, max_new_tokens=10):
    prompt = f"Lyrics: {lyrics}\nGenre:"
    inputs = llama_tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    outputs = llama_model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    generated = llama_tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return generated.strip().split("\n")[0]

In [ ]:
# MODEL 2: RoBERTa-LSTM-GRU Genre Classifier

def classify_roberta(lyrics):
    inputs = roberta_tokenizer(lyrics, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = roberta_model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)[0]
        pred_id = torch.argmax(probs).item()
        return {
            "genre": roberta_id2label[pred_id],
            "confidence": probs[pred_id].item(),
        }

In [ ]:
# MODEL 3: TinyLLaMA Chord Generator

def generate_chords(lyrics):
    prompt = f"""### Input Lyrics:
{lyrics}

### Chord Progression:
"""
    inputs = chord_tokenizer(prompt, return_tensors="pt").to(chord_model.device)
    with torch.no_grad():
        outputs = chord_model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=200,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=chord_tokenizer.pad_token_id,
            eos_token_id=chord_tokenizer.eos_token_id
        )
    generated = chord_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated.split("### Chord Progression:")[1].strip() if "### Chord Progression:" in generated else generated.strip()

In [ ]:
def get_multiline_input():
    print("Paste full multi-line lyrics below (use \\n for line breaks):")
    raw_input = input("> ")
    return raw_input.replace("\\n", "\n").strip()

In [ ]:
def menu():
    print("\n What would you like to do?")
    print("1. Classify genre with LLaMA3")
    print("2. Classify genre with RoBERTa")
    print("3. Generate chord progression")
    print("4. Run all models")
    print("5. Enter new lyrics")
    print("6. Exit")
    return input("Select an option (1-6): ").strip()

In [ ]:
def run_session():
    lyrics = get_multiline_input()

    while True:
        choice = menu()

        if choice == "6":
            print(" Goodbye!")
            break
        elif choice == "5":
            lyrics = get_multiline_input()
        elif choice == "1":
            genre = classify_llama3(lyrics)
            print(f"\n LLaMA3 Genre: {genre}")
        elif choice == "2":
            result = classify_roberta(lyrics)
            print(f"\n RoBERTa Genre: {result['genre']} (Confidence: {result['confidence']:.4f})")
        elif choice == "3":
            chords = generate_chords(lyrics)
            print(f"\n Chord Progression:\n{chords}")
        elif choice == "4":
            genre1 = classify_llama3(lyrics)
            result2 = classify_roberta(lyrics)
            chords = generate_chords(lyrics)

            print(f"\n LLaMA3 Genre: {genre1}")
            print(f"\n RoBERTa Genre: {result2['genre']} (Confidence: {result2['confidence']:.4f})")
            print(f"\n Chord Progression:\n{chords}")
        else:
            print(" Invalid option. Please try again.")

In [ ]:
if __name__ == "__main__":
    run_session()

Paste full multi-line lyrics below (use \n for line breaks):
> I found a love for me Darling, just dive right in and follow my lead Well, I found a girl, beautiful and sweet Oh, I never knew you were the someone waiting for me  'Cause we were just kids when we fell in love Not knowing what it was I will not give you up this time But darling, just kiss me slow Your heart is all I own And in your eyes you're holding mine  Baby, I'm dancing in the dark With you between my arms Barefoot on the grass Listening to our favourite song When you said you looked a mess I whispered underneath my breath But you heard it, Darling, you look perfect tonight

 What would you like to do?
1. Classify genre with LLaMA3
2. Classify genre with RoBERTa
3. Generate chord progression
4. Run all models
5. Enter new lyrics
6. Exit
Select an option (1-6): 1


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



 LLaMA3 Genre: Pop

 What would you like to do?
1. Classify genre with LLaMA3
2. Classify genre with RoBERTa
3. Generate chord progression
4. Run all models
5. Enter new lyrics
6. Exit
Select an option (1-6): 2

 RoBERTa Genre: R&B (Confidence: 0.2152)

 What would you like to do?
1. Classify genre with LLaMA3
2. Classify genre with RoBERTa
3. Generate chord progression
4. Run all models
5. Enter new lyrics
6. Exit
Select an option (1-6): 3

 Chord Progression:
C|-------------------|-------------------|------------------|-------------------|
G|------------------|------------------|------------------|------------------|
D|------------------|------------------|------------------|------------------|
A|------------------|------------------|------------------|------------------|
E|------------------|------------------|------------------|------------------|

### Chord Progressions:

G|-------------------|-------------------|------------------|
D|------------------|------------------|-------

In [ ]:
!pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 135.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.1 MB/s eta 0:00:00


In [ ]:
import gradio as gr

def run_models(lyrics, task):
    results = {}

    if task == "LLaMA3 Genre Classification" or task == "Run All":
        results["LLaMA3 Genre"] = classify_llama3(lyrics)

    if task == "RoBERTa Genre Classification" or task == "Run All":
        roberta_out = classify_roberta(lyrics)
        results["RoBERTa Genre"] = f"{roberta_out['genre']} (Confidence: {roberta_out['confidence']:.4f})"

    if task == "Chord Progression Generation" or task == "Run All":
        chords = generate_chords(lyrics)
        results["Chord Progression"] = chords

    return results


In [ ]:
tasks = [
    "LLaMA3 Genre Classification",
    "RoBERTa Genre Classification",
    "Chord Progression Generation",
    "Run All"
]

gr.Interface(
    fn=run_models,
    inputs=[
        gr.Textbox(lines=10, label="Enter Song Lyrics", placeholder="Paste your lyrics here..."),
        gr.Dropdown(choices=tasks, label="Choose Task")
    ],
    outputs=gr.JSON(label="Model Outputs"),
    title="Lyrics Genre & Chord Generator",
    description="Uses LLaMA3, RoBERTa, and TinyLLaMA to classify lyrics and generate chords."
).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8c1b539c7ba83200cb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
